# Testing + mini tutorial of pyvisa to control the oscilliscope

Notes
- To check what variables are available to call or change on the oscilliscope, open XStreamBrowser on the oscilliscope. The tree in the browser gives you the path and the information on particular variable tells you the type, whether you can read/write, and possible ingputs (Range/Helpstring)
    - Example: app.acquisition.horizontal.maximize has the options "FixedSampleRate" and "SetMaximumMemory" (still not sure what this means, but it didn't work until I used one of these)

In [1]:
import pyvisa
import numpy as np

In [2]:
# Before running, connect the laptop to the oscilloscope through the USBTMC
rm = pyvisa.ResourceManager()
rm.list_resources() # use this to grab the instrument in the next step

('USB0::0x05FF::0x1023::4206N21552::INSTR', 'ASRL3::INSTR', 'ASRL10::INSTR')

In [3]:
# Grabs the instrument and prints
scope = rm.open_resource('USB0::0x05FF::0x1023::4206N21552::INSTR')

# Message for “what are you?”
scope.query('*IDN?')

'*IDN LECROY,WAVERUNNER8054,LCRY4206N21552,8.5.1\n'

In [4]:
# Change the vertical scale and horizontal scale of channel 1
# (good way to check if we can send commands to the scope)
scope.write(r"""vbs 'app.Acquisition.C1.VerScale = 1.0' """)
scope.write(r"""vbs 'app.Acquisition.Horizontal.HorScale = 500e-6' """)

(53, <StatusCode.success: 0>)

In [5]:
# Print the vertical offset for channel 1 
# (good way to check that we can recieve info from the scope)
(scope.query(r"""vbs? 'return=app.Acquisition.C1.VerScale' """))

'VBS 1\n'

In [6]:
# Waits 5 seconds after oscilliscope is idle before 
# setting up new acquisition
scope.timeout = 5000
scope.clear()
r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)

In [7]:
# Set up aquisition by stopping trigger 
trigger_level = .10
scope.write(r"""vbs 'app.acquisition.triggermode = "stopped" ' """)
scope.write(r"""vbs 'app.acquisition.trigger.edge.level = %f ' """ % trigger_level)
scope.write(r"""vbs 'app.acquisition.triggermode = "single" ' """)
scope.write(r"""vbs 'app.acquisition.horizontal.maximize = "FixedSampleRate" ' """)

(65, <StatusCode.success: 0>)

In [8]:
# Set up aquisition by stopping trigger 
trigger_level = .2
scope.write(r"""vbs 'app.acquisition.triggermode = "stopped" ' """)
# Set up trigger (in this case we trigger on the C2 which has the trigger for the input waveform to the LED)
scope.write(r"""vbs 'app.acquisition.trigger.edge.source = "C2" ' '""")
scope.write(r"""vbs 'app.acquisition.trigger.C2level = %f ' """ % trigger_level)
scope.write(r"""vbs 'app.acquisition.triggermode = "single" ' """)
scope.write(r"""vbs 'app.acquisition.horizontal.maximize = "FixedSampleRate" ' """)

(65, <StatusCode.success: 0>)

In [9]:
wait_time = 0.03
# Set up Measurements, clearing previous measurements
scope.write(r"""vbs 'app.clearsweeps ' """)
scope.write(r"""vbs 'app.measure.clearall ' """)
scope.write(r"""vbs 'app.measure.clearsweeps ' """)

# Setup averaging waveform
scope.write(r"""vbs 'app.math.f1.operator1 = "intg" ' """)
scope.write(r"""vbs 'app.math.f1.source1 = "C1" ' """)

# Setup measurement of peak to peak voltage based on average waveform
scope.write(r"""vbs 'app.measure.showmeasure = true ' """)
scope.write(r"""vbs 'app.measure.statson = true ' """)
scope.write(r"""vbs 'app.measure.p1.view = true ' """)
scope.write(r"""vbs 'app.measure.p1.paramengine = "maximum" ' """ )
scope.write(r"""vbs 'app.measure.p1.source1 = "F1" ' """)

# Sweep
num_sweeps = 10
integrals = []
for i in range(0,num_sweeps):
    r = scope.query(r"""vbs? 'return=app.acquisition.acquire( %f , True ) ' """ %wait_time)
    r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(0.003)' """)
    integral = scope.query(r"""vbs? 'return=app.measure.p1.out.result.value' """)
    integrals.append(integral)
    if r==0:
        print("Time out from WaitUntilIdle, return = {0}".format(r))


integrals

['VBS 0.006063251116913\n',
 'VBS 0.006063526355287\n',
 'VBS 0.00606359193345\n',
 'VBS 0.006063377241905\n',
 'VBS 0.006063500587788\n',
 'VBS 0.006063510433744\n',
 'VBS 0.006063595313877\n',
 'VBS 0.00606295467006\n',
 'VBS 0.006063243772476\n',
 'VBS 0.006063602057496\n']

In [10]:
def MeasureIntegral():
    """
    Measures the integral of the sipm waveform coming from the oscilliscope and
    returns an array of the maxes for a given number of aquisitions.
    """
    force_trigger_rate = 0.1 # s
    trigger_level = 0.890 # V
    
    # Set up aquisition by stopping trigger 
    scope.write(r"""vbs 'app.acquisition.triggermode = "stopped" ' """)
    # Set up trigger (in this case we trigger on the C2 which has the trigger for the input waveform to the LED)
    scope.write(r"""vbs 'app.acquisition.trigger.edge.source = "C2" ' '""")
    scope.write(r"""vbs 'app.acquisition.trigger.C2level = %f ' """ % trigger_level)
    scope.write(r"""vbs 'app.acquisition.triggermode = "single" ' """)
    scope.write(r"""vbs 'app.acquisition.horizontal.maximize = "FixedSampleRate" ' """)

    # Set up Measurements, clearing previous measurements
    scope.write(r"""vbs 'app.clearsweeps ' """)
    scope.write(r"""vbs 'app.measure.clearall ' """)
    scope.write(r"""vbs 'app.measure.clearsweeps ' """)

    # Setup averaging waveform
    scope.write(r"""vbs 'app.math.f1.operator1 = "intg" ' """)
    scope.write(r"""vbs 'app.math.f1.source1 = "C1" ' """)

    # Setup measurement of peak to peak voltage based on average waveform
    scope.write(r"""vbs 'app.measure.showmeasure = true ' """)
    scope.write(r"""vbs 'app.measure.statson = true ' """)
    scope.write(r"""vbs 'app.measure.p1.view = true ' """)
    scope.write(r"""vbs 'app.measure.p1.paramengine = "maximum" ' """ )
    scope.write(r"""vbs 'app.measure.p1.source1 = "F1" ' """)

    # Sweep
    integrals = []
    for i in range(0,NUM_SWEEPS):
        r = scope.query(r"""vbs? 'return=app.acquisition.acquire( %f , True ) ' """ %force_trigger_rate)
        r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(0.003)' """)
        integral = scope.query(r"""vbs? 'return=app.measure.p1.out.result.value' """)
        integrals.append(integral)
        if r==0:
            print("Time out from WaitUntilIdle, return = {0}".format(r))

    integrals = np.array([ConvertToFloat(integral) for integral in integrals])
    
    return integrals/np.mean(integrals)

def ConvertToFloat(val):
    """
    Extracts a float from a complex string
    """
    for token in val.split():
        floatval = ''
        try:
           floatval = float(token)
        except ValueError:
            continue
        if type(floatval)==float:
            return floatval
    return np.nan

In [11]:
NUM_SWEEPS = 10
MeasureIntegral()

array([1.00002783, 0.99995337, 0.9999837 , 1.00002277, 0.99997953,
       1.00000296, 1.00005582, 0.999982  , 1.00007336, 0.99991867])

In [25]:
# Make sure to close the connection (in teh reverse order of setting it up)
scope.close()
rm.close()